In [5]:
import requests
# Usage
file_path = "hh-14.tar.xz"  # replace with your local path
repo_url = "https://mygit.th-deg.de/tcg/gelap/-/raw/master/" + file_path + "?ref_type=heads"  # replace with your repo URL

# Download the file
response = requests.get(repo_url)
with open(file_path, 'wb') as f:
    f.write(response.content)
print("Downloaded", file_path)


Downloaded hh-14.tar.xz


In [ ]:
import os
import pandas as pd

household_path = 'C://Users//seho04//Documents//GitHub//thingsboard-replayer//example_data//hh_14'
labels = { 
            "label_001.csv": "thermomix",
            "label_002.csv": "toaster",
            "label_003.csv": "coffee_machine",
            "label_004.csv": "electric_kettle",
            "label_005.csv": "microwave",
            "label_006.csv": "radio",
            "label_007.csv": "washing_machine",
            "label_008.csv": "vacuum_cleaner",
            "label_009.csv": "television",
            "label_010.csv": "charger"}

In [ ]:
smartmeter = pd.read_csv(os.path.join(household_path, "smartmeter.csv"))
smartmeter = smartmeter.set_index("time")
smartmeter.index = pd.to_datetime(smartmeter.index, unit='ms')
# shorten the timestamp to seconds
smartmeter.index = smartmeter.index.floor('s')
# add all the power columns together
smartmeter["power"] = smartmeter["power1"] + smartmeter["power2"] + smartmeter["power3"]
smartmeter = smartmeter.rename(columns={"power": "smartmeter"})
smartmeter = smartmeter.drop(columns=["power1", "power2", "power3"])
smartmeter.to_csv(os.path.join(os.path.join(household_path, 'edited'), "smartmeter.csv"))
smartmeter = smartmeter.drop(columns=["smartmeter"])

for label in labels:
    print(label, labels[label])
    # for device in pd.read_csv(os.path.join(household_path, label), chunksize=10000):
    device = pd.read_csv(os.path.join(household_path, label))
    device = device.rename(columns={"time_request": "time"})
    device = device.drop(columns=["time_reply"])
    device.set_index("time")
    device.index = pd.to_datetime(device.index, unit='ms')
    device.index = device.index.floor('s')
    
    # add the power column to the smartmeter dataframe at the matching timestamp
    smartmeter = pd.merge(smartmeter, device, how='left', left_index=True, right_index=True) 
    
    smartmeter = smartmeter.rename(columns={"power": labels[label]})
    smartmeter = smartmeter.drop(columns=["time"])
    smartmeter.to_csv(os.path.join(os.path.join(household_path, 'edited'),labels[label]+".csv"))
    smartmeter = smartmeter.drop(columns=[labels[label]])

In [ ]:
# loop over folder and merge all csv files into one csv file
files = os.listdir(os.path.join(household_path, 'edited'))
df = pd.read_csv(os.path.join(os.path.join(household_path, 'edited'), files[0]))
# rename the Unamed column to "time"
df = df.rename(columns={"Unnamed: 0": "time"})
# set the timestamp as the index
df = df.set_index("time")
for file in files[1:]:
    device = pd.read_csv(os.path.join(os.path.join(household_path, 'edited'), file))
    # copy all collumnds except the index column to the new dataframe
    df = pd.concat([df, device.iloc[:, 1:]], axis=1)
     
df.to_csv(os.path.join(household_path, "merged.csv"))